In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={
    "rmse_threshold": 15
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, NotebookSubstepRunResult, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "data_prep", ENTITY_NAME: "df_X_test" },
        { STEP_NAME: "data_prep", ENTITY_NAME: "df_Y_test" },
        { STEP_NAME: "model_train", ENTITY_NAME: "bento_service" },
    ],
    outputs = 
    [
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
#5 read inputs 
prev_step_inputs = substep.inputs(step_name="data_prep")
bento_step_inputs = substep.inputs(step_name="model_train")

df_X_test = spark.read.parquet(prev_step_inputs.df_X_test).to_pandas_on_spark()
df_Y_test = spark.read.parquet(prev_step_inputs.df_Y_test).to_pandas_on_spark()

In [ ]:
#6 load a bentoservice
from sinara.bentoml import load_bentoservice
from sklearn.metrics import mean_squared_error
bento_serv = load_bentoservice(bento_step_inputs.bento_service)

In [ ]:
#7 evaluate model, check model quality on test dataset
predictions = bento_serv.predict(df_X_test)
rmse = mean_squared_error(df_Y_test.values, predictions, squared=False)
print("The root mean squared error (RMSE) on test set: {:.4f}".format(rmse))

In [ ]:
#8 check eval result
rmse_threshold = substep_params["rmse_threshold"]
if rmse > rmse_threshold:
    raise Exception(f'RMSE is {rmse}, more than acceptable value of {rmse_threshold}')

In [ ]:
#8 stop spark
SinaraSpark.stop_session()